In [1]:
import os
import torch
from dataset.dataloader import DataloaderMode, create_dataloader
from model.model import Model
from model.model_arch import Net_arch
from utils.utils import get_pytorch_kobert_model
from utils.utils import calc_metric
from omegaconf import OmegaConf

model_checkpoint_name = "chkpt/dynamic lr + 512 truncation for val set_2022-12-12 00:51:44_7508.pt"
dataset_file_name = "dataset/meta/KorQuAD_v1.0_dev.json"
config_file_name = "config/job/train/default.yaml"

In [2]:
cfg = OmegaConf.load(config_file_name)
cfg.working_dir = os.getcwd()
cfg.job_logging_cfg = {'version': 1, 'formatters': {'simple': {'format': '%(message)s'}, 'detailed': {'format': '[%(asctime)s][%(name)s][%(levelname)s] - %(message)s'}}, 'handlers': {'console': {'class': 'logging.StreamHandler', 'formatter': 'detailed', 'stream': 'ext://sys.stdout'}, 'file': {'class': 'logging.FileHandler', 'formatter': 'detailed', 'filename': 'trainer.log'}}, 'root': {'level': 'INFO', 'handlers': ['console', 'file']}, 'disable_existing_loggers': False}
cfg.test.batch_size = 256
print(cfg)

{'name': 'NLP Final Project', 'working_dir': '/data/project/rw/andrew/snu/snu-nlp-2022', 'device': 'cuda', 'random_seed': None, 'start_time': None, 'num_epoch': 8, 'data': {'train_file': 'dataset/meta/KorQuAD_v1.0_train.json', 'test_file': 'dataset/meta/KorQuAD_v1.0_dev.json', 'file_format': '*.file_extension', 'divide_dataset_per_gpu': True}, 'train': {'num_workers': 4, 'batch_size': 32, 'optimizer': {'mode': 'adam', 'adam': {'lr': 5e-05, 'betas': [0.9, 0.999]}}, 'scheduler': {'max_lr': '${train.optimizer.adam.lr}', 'min_lr': 2e-05}}, 'test': {'num_workers': 4, 'batch_size': 256}, 'model': None, 'dist': {'master_addr': 'localhost', 'master_port': '12355', 'mode': 'nccl', 'gpus': 0, 'timeout': 30}, 'log': {'use_tensorboard': False, 'use_wandb': True, 'wandb_init_conf': {'name': '${name}', 'entity': 'snu-nlp', 'project': 'KorBert'}, 'summary_interval': 1, 'chkpt_interval': 1, 'chkpt_dir': 'chkpt'}, 'load': {'wandb_load_path': None, 'network_chkpt_path': None, 'strict_load': False, 'resu

In [3]:
net_arch = Net_arch(cfg)
model = Model(cfg, net_arch, None)
model.load_network(torch.load(model_checkpoint_name, map_location=torch.device(cfg.device)))

using cached model. /data/project/rw/andrew/snu/snu-nlp-2022/.cache/kobert_v1.zip
using cached model. /data/project/rw/andrew/snu/snu-nlp-2022/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [4]:
test_loader = create_dataloader(cfg, DataloaderMode.test, 0)

using cached model. /data/project/rw/andrew/snu/snu-nlp-2022/.cache/kobert_v1.zip
using cached model. /data/project/rw/andrew/snu/snu-nlp-2022/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [5]:
def get_vocab():
    _, vocab = get_pytorch_kobert_model()
    return vocab

vocab = get_vocab()

using cached model. /data/project/rw/andrew/snu/snu-nlp-2022/.cache/kobert_v1.zip
using cached model. /data/project/rw/andrew/snu/snu-nlp-2022/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [7]:
def eval_model(cfg, model, test_loader, n_batches=-1):
    model.net.eval()
    total_test_loss = 0
    test_loop_len = 0
    texts = dict()
    with torch.no_grad():
        for data in test_loader:
            if n_batches > -1 and test_loop_len >= n_batches:
                break
            print(f"Processing batch {test_loop_len}...")
            model_input, model_target, qa_id = data[:3], data[3:5], data[5]
            output = model.inference(model_input)

            # Calculate QA metrics
            output = output.mT  # [batch, num_words, 2] -> [batch, 2, num_words]
            output = torch.softmax(output, dim=-1)
            # print(model_target)
            # print(vocab.decode(model_input[0][0].tolist()))
            # for i in output[0][0].tolist(): print("%.2f " % i, end="")
            # print("")
            # for i in output[0][1].tolist(): print("%.2f " % i, end="")
            # print("")
            # print(vocab.decode(model_input[0][0][24:30].tolist()))
            start = torch.argmax(output[..., 0, :], dim=-1)  # [batch, 2, num_words] -> [batch]
            batch_size = model_input[0].shape[0]
            for i in range(batch_size):
                input_text = model_input[0][i]
                pred_start = start[i]
                pred_end = pred_start + torch.argmax(output[i, 1, pred_start:], dim=-1)
                pred_text = input_text[pred_start : pred_end + 1].tolist()
                decoded_text = vocab.decode(pred_text)
                texts[qa_id[i]] = decoded_text

            test_loop_len += 1

        total_test_loss /= test_loop_len
        metric = calc_metric(cfg, texts)
    return texts, metric

texts, metric = eval_model(cfg, model, test_loader)
metric

Processing batch 0...
Processing batch 1...
Processing batch 2...
Processing batch 3...
Processing batch 4...
Processing batch 5...
Processing batch 6...
Processing batch 7...
Processing batch 8...
Processing batch 9...
Processing batch 10...
Processing batch 11...
Processing batch 12...
Processing batch 13...
Processing batch 14...
Processing batch 15...
Processing batch 16...
Processing batch 17...
Processing batch 18...
Processing batch 19...
Processing batch 20...
Processing batch 21...


{'exact_match': 53.41184620713543, 'f1': 72.03657918106603}